# Optimize data for analysis: Learnings from AWS Open Data

[![Open In Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/pschmied/opn201/blob/main/better-know-a-dataset/python/example.ipynb)

(https://github.com/pschmied/opn201)

## Contacts / links
 - Peter Schmiedeskamp - Statistical & Regulatory Lead (pws AT amazon), AWS Open Data
 - AWS Open Data (opendata AT amazon / https://opendata.aws)
 - Registry of Open Data (https://registry.opendata.aws)

## Amazon Simple Storage Service (S3)

Three main tuning knobs:
 1. Object size
 2. Object name
 3. Object format (including compression)

### Object size
 - Too small: many requests (i.e. >= 1 per object); implications for listing 🙀
 
 ```
 $ aws s3 ls s3://fictitious-bucket-of-a-billion-1KiB-objects
 2022-05-17 18:10:49  1024 obj1.txt
 2022-05-17 18:10:50  1024 obj2.txt
 2022-05-17 18:10:51  1024 obj3.txt
 [...]
 2022-05-18 19:14:04  1024 obj1000000000.txt
 ```
 
 - Too big: may force customers to download more data than they need 🤔
 
 ```
 $ aws s3 ls s3://fictitious-bucket-of-a-1TiB-object
 2022-05-17 18:10:49  1.1e+12 obj1.txt
 ```
 
 - Just right?: It depends 🤷
 ```
 $ aws s3 ls s3://fictitious-bucket-of-a-thousand-1GiB-objects
 2022-05-17 18:10:49  1.074e+9 obj1.txt
 2022-05-17 18:10:50  1.074e+9 obj2.txt
 2022-05-17 18:10:51  1.074e+9 obj3.txt
 [...]
 2022-05-18 19:14:04  1.074e+9 obj1000.txt
 ```
 

 ### Object name
 - Theory: S3 is a flat namespace; buckets have no concept of directories; '/' is [not special](https://docs.aws.amazon.com/AmazonS3/latest/userguide/object-keys.html)
 - Practice: '/' is, by convention, used as a delimiter by many services including Athena
 
 Recommend using "Hive-style partition" naming conventions [as outlined in Athena documentation](https://docs.aws.amazon.com/athena/latest/ug/partitions.html) (even if you won't use Athena)

In [1]:
# Install the 'tree' utility if not already installed
!conda install -y tree

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [2]:
!aws --no-sign-request s3 ls --recursive \
   s3://ookla-open-data/shapefiles/performance/type=mobile/year=2021/  \
   | awk ' { print $4 } ' \
   | tree --noreport --fromfile

.
└── shapefiles
    └── performance
        └── type=mobile
            └── year=2021
                ├── quarter=1
                │   └── 2021-01-01_performance_mobile_tiles.zip
                ├── quarter=2
                │   └── 2021-04-01_performance_mobile_tiles.zip
                ├── quarter=3
                │   └── 2021-07-01_performance_mobile_tiles.zip
                └── quarter=4
                    └── 2021-10-01_performance_mobile_tiles.zip


In [3]:
# Also works, but may cause users some additional effort
!aws --no-sign-request s3 ls --recursive \
   s3://noaa-goes16/ABI-L2-RRQPEF/2020/ 2>/dev/null \
   | awk ' { print $4 } ' 2>/dev/null \
   | head -n 18 \
   | tree --noreport --fromfile

.
└── ABI-L2-RRQPEF
    └── 2020
        └── 001
            ├── 00
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s20200010000216_e20200010009524_c20200010010034.nc
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s20200010010216_e20200010019524_c20200010020035.nc
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s20200010020216_e20200010029524_c20200010030028.nc
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s20200010030216_e20200010039524_c20200010040050.nc
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s20200010040216_e20200010049524_c20200010050040.nc
            │   └── OR_ABI-L2-RRQPEF-M6_G16_s20200010050216_e20200010059524_c20200010100027.nc
            ├── 01
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s20200010100216_e20200010109524_c20200010110037.nc
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s20200010110216_e20200010119524_c20200010120042.nc
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s20200010120216_e20200010129524_c20200010130033.nc
            │   ├── OR_ABI-L2-RRQPEF-M6_G16_s2020001013021

### Object format
Theory:
 - Choose formats that support range requests (or [S3 Select... 🤔 or both!? 🤯](https://docs.aws.amazon.com/AmazonS3/latest/userguide/selecting-content-from-objects.html)? )
 - Use compression; don't use archives
 
Practice: gzip / NOT tar, ndjson.gz, csv.gz, COG, Parquet, ORC, Zarr, OH MY! 

## Tabular data with Athena

Athena needs three things: data, schema, (often) index of partitions.

Data are stored as objects in S3, mapped to a relational schema via Data Definition Language (DDL). Partitions are represented as dimensions (columns) in the schema, and are either enumerated in AWS Glue catalog (or external Hive metastore), or inferred via Athena Partition Projection. Note Partition Projection is Athena-only, and doesn't apply to e.g. Glue.

CSV

```sql
CREATE EXTERNAL TABLE `sentinel_s1_l1c_inventory_csv`(
  `bucket` string COMMENT 'from deserializer', 
  `key` string COMMENT 'from deserializer', 
  `size` string COMMENT 'from deserializer', 
  `last_modified_date` string COMMENT 'from deserializer')
PARTITIONED BY ( 
  `dt` string)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://sentinel-inventory/sentinel-s1-l1c/sentinel-s1-l1c-inventory/hive'
```

ORC

```sql
CREATE EXTERNAL TABLE `sentinel_s1_l1c_inventory_orc`(
  `bucket` string, 
  `key` string, 
  `size` bigint, 
  `last_modified_date` timestamp)
PARTITIONED BY ( 
  `dt` string)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.orc.OrcSerde' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://sentinel-inventory/orc/sentinel-s1-l1c/sentinel-s1-l1c-inventory-orc/hive'
```

### Query performance ORC vs. CSV
ORC
```sql
SELECT AVG(SIZE) FROM sentinel_s1_l1c_inventory_orc
```
(Run time: ~24 sec; Data scanned: ~120 MB)

vs. CSV

```sql
SELECT AVG(SIZE) FROM sentinel_s1_l1c_inventory_orc
```
(Run time: ~10 min; Data scanned: ~165 GB)

Something interesting is happening here... It's not necessarily easy, but Hive-convention partitions can be grafted on after the fact!

In [4]:
!aws --no-sign-request s3 ls --recursive \
   s3://sentinel-inventory/orc/sentinel-s1-l1c/sentinel-s1-l1c-inventory-orc/hive 2>/dev/null \
   | awk ' { print $4 } ' 2>/dev/null \
   | head -n5 \
   | tree --noreport --fromfile

.
└── orc
    └── sentinel-s1-l1c
        └── sentinel-s1-l1c-inventory-orc
            └── hive
                ├── dt=2021-01-26-00-00
                │   └── symlink.txt
                ├── dt=2021-01-27-00-00
                │   └── symlink.txt
                ├── dt=2021-01-28-00-00
                │   └── symlink.txt
                ├── dt=2021-01-29-00-00
                │   └── symlink.txt
                └── dt=2021-01-30-00-00
                    └── symlink.txt


In [5]:
!aws --no-sign-request s3 cp s3://sentinel-inventory/orc/sentinel-s1-l1c/sentinel-s1-l1c-inventory-orc/hive/dt=2021-01-26-00-00/symlink.txt - \
    | head -n4

s3://sentinel-inventory/orc/sentinel-s1-l1c/sentinel-s1-l1c-inventory-orc/data/661f4db0-398a-454c-b810-7856a9530067.orc
s3://sentinel-inventory/orc/sentinel-s1-l1c/sentinel-s1-l1c-inventory-orc/data/5e536e18-735d-4181-ab84-4e4a1f43f973.orc
s3://sentinel-inventory/orc/sentinel-s1-l1c/sentinel-s1-l1c-inventory-orc/data/399d7ac9-40d6-4de7-94fc-2eb233d8b6c9.orc
s3://sentinel-inventory/orc/sentinel-s1-l1c/sentinel-s1-l1c-inventory-orc/data/7d21fdd0-9431-4212-91a3-b1ac3158550e.orc


Note: Examples here assume write few / read many. It's not unusual for tabular data to be added (`UPSERT`-ed) to regularly. Check out [Apache Hudi](https://docs.aws.amazon.com/athena/latest/ug/querying-hudi.html) if this sounds like you.

## Specialty formats / beyond tabular - Genomic Data

Genomic variant call files (VCFs): https://registry.opendata.aws/ilmn-dragen-1kgp/

...enhance! After initial header rows, data are tabular.

In [6]:
!aws s3 cp --no-sign-request s3://1000genomes-dragen/data/precisionFDA/hg38_altaware_nohla-cnv-anchored/HG004/sv/results/variants/diploidSV.vcf.gz - 2>/dev/null \
    | zcat 2>/dev/null \
    | head -n10000 \
    | tail -n1

chr10	65272628	MantaDEL:104013:0:0:0:0:0	ATTTAAGGTGCACAATAGAGAAGTCAGTTGTTAAAAAACAGAGAATGTGGACTTTGGAGATGGTATTGTGTAGTCAAGAGAGCCAACAATGAGTGAGATAATTTTCTGTATTACCTTTGTATCCTTGTACAAGTTTTAATTTCTCTAAGCCTAGCTTTCTCATCTGTAAATCTGGCATAATATTTATTATGATTAAGTGAAACAATGCTGATAAACTCTTTTTGTAGAAGCTGACCATAGAAAGTACTTAAAACAAAACATGTTTTAATATTATTTCCAAATGTCAAATAGCCAAATAAATTCTCCCAGATATTGTGTGACCAAATGAAACATAATGTTAATTGATACATAGATTATGACTATAAAACTTTTGTCAGAATAAGATGAGCTCTTACCCAGGTTAGGTTTACAAGGCA	AGAACACAGAAAGACCTAAACT	999	PASS	END=65273043;SVTYPE=DEL;SVLEN=-415;CIGAR=1M21I415D	GT:FT:GQ:PL:PR:SR	1/1:PASS:100:999,103,0:0,14:0,28


This implies that at least some potential for conversion to columnar formats like Parquet and ORC.

Side note, Athena is a passable distributed `grep` even if you take a naive approach to text-based data.

```sql


```

Recommended reading: [Data Lake as Code](https://aws.amazon.com/blogs/startups/a-data-lake-as-code-featuring-chembl-and-opentargets/)

## Specialty formats / beyond tabular - Geospatial
Vector formats: is tabular; geometries are just a special column type; e.g. Parquet works well / `ST_*` operations are supported in Athena.

Example, OSM Daylight in Parquet:
 - https://registry.opendata.aws/daylight-osm/
 - https://gist.github.com/jenningsanderson/3e42a99dcb8f760038ad8aa47ea38ce8


Raster formats - imagery with geospatial metadata. We are fond of Cloud-Optimized GeoTIFF:

 - STAC/COG example(s) - https://registry.opendata.aws/deafrica-sentinel-2/
     - [Blog Post on Using STAC with S-2](https://docs.digitalearthafrica.org/en/latest/sandbox/notebooks/Frequently_used_code/Downloading_data_with_STAC.html)
 - https://registry.opendata.aws/usgs-landsat/ and [blog post](https://pystac.readthedocs.io/en/stable/tutorials/creating-a-landsat-stac.html)
 - https://registry.opendata.aws/sentinel-2-l2a-cogs/ with [blog post](https://www.element84.com/blog/sentinel-2-cloud-optimized-geotiffs-now-available-on-aws-registry-of-open-data) and [another blog post](https://aws.amazon.com/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/)

## Specialty formats / beyond tabular - Array / Matrix

Current recommendation for customers is [Zarr, which came from the Python scientific community](https://zarr.readthedocs.io/en/stable/), but has now been replicated in e.g. [Julia](https://github.com/JuliaIO/Zarr.jl) and [Rust](https://github.com/sci-rs/zarr) to varying degrees of completeness.

Noteworthy dataset examples:
- https://registry.opendata.aws/mur/ MUR Level 4 SST dataset in Zarr format. The zarr/ directory contains a zarr store chunked along the dimensions (time, lat, lon).


- https://registry.opendata.aws/nwm-archive/ with 3 Zarr Tutorials: 
    - [Explore Repository of Tutorials on National Water Model V2.1 Retrospective Dataset in Zarr](https://github.com/NCAR/rechunk_retro_nwm_v21/tree/main/notebooks)
    - [Explore the National Water Model V2.0 Retrospective in Zarr](https://nbviewer.jupyter.org/gist/rsignell-usgs/d3dfaf3cd3d8b39894a69b22127dfe38)
    - [Explore the National Water Model V2.1 Retrospective Dataset in Zarr](https://github.com/NCAR/rechunk_retro_nwm_v21)


- https://registry.opendata.aws/noaa-hrrr-pds/ with:
    - [The HRRR Zarr Archive Managed by MesoWest](https://mesowest.utah.edu/html/hrrr/)
    - [HRRR-B Python package: download and read HRRR grib2 files](https://github.com/blaylockbk/HRRR_archive_download)


See also [Kerchunk](https://fsspec.github.io/kerchunk/index.html) which is an emerging approach to working with chunked array data.